In [36]:
import pandas as pd
import hopsworks
import os
import requests

In [37]:
player_stats_gw_url = "https://fantasy.premierleague.com/api/event/{gw}/live/"
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"

In [ ]:
try:
    with open('./hopsworks/hopsworks-api-key.txt', 'r') as file:
        os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
except:
    print("In production mode")

project = hopsworks.login()
print(project.description)

In [ ]:
general_info = requests.get(bootstrap_url).json()
general_stats = pd.DataFrame(general_info["elements"])
general_stats

In [40]:
fs = project.get_feature_store()

In [ ]:
player_fg = fs.get_feature_group("player_features")

In [ ]:
prev_gameweek = player_fg.select(["gameweek"]).read().max()

prev_gameweek = next(
    (gw for gw in general_info['events'] if gw['id'] == int(prev_gameweek)), None
)
prev_finished = prev_gameweek['finished']
prev_finished

In [43]:
lagged_features = ['minutes', 'goals_scored', 'assists', 'clean_sheets',
                   'goals_conceded', 'own_goals', 'penalties_saved',
                   'penalties_missed', 'yellow_cards', 'red_cards', 'saves',
                   'bonus', 'bps', 'influence', 'creativity', 'threat',
                   'ict_index', 'starts', 'expected_goals', 'expected_assists',
                   'expected_goal_involvements', 'expected_goals_conceded',
                   'in_dreamteam', 'total_points', 'selected', 'transfers_balance',
                   'value', 'was_home', 'opponent_team']

In [44]:
player_details_url = "https://fantasy.premierleague.com/api/element-summary/{element_id}"

In [45]:
if prev_finished:
    prev_stats = player_stats_gw_url.format(gw=prev_gameweek["id"])
    prev_stats = requests.get(prev_stats).json()
    prev_stats_df = pd.DataFrame(prev_stats["elements"]).drop(columns=["explain", "modified"])
    prev_stats_df = pd.concat(
        [prev_stats_df.drop(columns=["stats"]), prev_stats_df["stats"].apply(pd.Series)],
        axis=1
    )
    prev_stats_df.rename(columns={"total_points": "points"}, inplace=True)

In [ ]:
if prev_finished:
    existing_data = player_fg.read()

    existing_data = existing_data.set_index('id')
    prev_stats_df = prev_stats_df.set_index('id')

    # Update the "points" column for the specified gameweek
    existing_data.loc[existing_data['gameweek'] == prev_gameweek["id"], 'points'] = prev_stats_df['points']

    # Reset index if needed
    existing_data = existing_data.reset_index()
    prev_stats_df = prev_stats_df.reset_index()

    # Save the updated data back to the feature group
    player_fg.insert(existing_data[existing_data["gameweek"] == prev_gameweek["id"]])
    print(f"Updated player points for gameweek {prev_gameweek['id']}")

In [ ]:
if prev_finished:

    for i in range(1, len(prev_stats_df['id'].unique()) + 1):

        player_details_url_formatted = player_details_url.format(element_id=i)
        player_details = requests.get(player_details_url_formatted).json()
        player_details_history_df = pd.DataFrame(player_details["history"])

        total_points = player_details_history_df["total_points"].sum()

        row = player_details_history_df[player_details_history_df['round'] == prev_gameweek["id"]].iloc[0]

        prev_stats_df.loc[prev_stats_df['id'] == i, 'total_points'] = total_points
        prev_stats_df.loc[prev_stats_df['id'] == i, 'selected'] = row['selected']
        prev_stats_df.loc[prev_stats_df['id'] == i, 'transfers_balance'] = row['transfers_balance']
        prev_stats_df.loc[prev_stats_df['id'] == i, 'value'] = row['value']
        prev_stats_df.loc[prev_stats_df['id'] == i, 'was_home'] = row['was_home']
        prev_stats_df.loc[prev_stats_df['id'] == i, 'opponent_team'] = row['opponent_team']


    prev_stats_df.rename(columns={col: f"prev_{col}" for col in prev_stats_df.columns if col not in ["id", "gameweek", "points"]}, inplace=True)

    prev_stats_df = prev_stats_df.dropna(subset=[f'prev_{col}' for col in lagged_features] + ['points'])


    prev_stats_df = prev_stats_df.astype({col: 'float64' for col in prev_stats_df.columns if col not in ["id", "gameweek", "points"]})


    prev_stats_df["gameweek"] = prev_gameweek["id"] + 1

    # Merge the general stats DataFrame with the all_gameweeks_df DataFrame
    prev_stats_df = prev_stats_df.merge(general_stats[["id","element_type"]], on="id", how="left")
    
    fs = project.get_feature_store()

    player_fg = fs.get_feature_group(
        name="player_features",
        version=1,
    )

    player_fg.insert(prev_stats_df)

    print("Inserted data for gameweek", prev_gameweek["id"] + 1)